In [1]:
%%file integrate.py
from datetime import datetime

def integrate(f, left, right, step):
    acc, cur = 0, left
    start = datetime.now()
    while cur < right:
        acc += f(cur) * min(right - cur, step)
        cur += step
    fin = datetime.now()
    return acc, start, fin, left, right

Overwriting integrate.py


In [2]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from integrate import integrate

def integrate_async(f, a, b, *, n_jobs=1, n_iter=1000, process=False, log=False):
    logs, ans = [], []
    step = (b - a) / n_iter
    num = (b - a) / n_jobs
    cur = a
    
    if process:
        with ProcessPoolExecutor(max_workers=n_jobs) as executor:
            res = 0
            for i in range(n_jobs):
                nxt = cur + num
                ans.append(executor.submit(integrate, f, cur, min(nxt, b), step))
                cur = nxt
            for task in ans:
                acc, start_time, end_time, left, right = task.result()
                res += acc
                logs.append((start_time, end_time, left, right))
    else:
        with ThreadPoolExecutor(max_workers=n_jobs) as executor:
            res = 0
            for i in range(n_jobs):
                nxt = cur + num
                ans.append(executor.submit(integrate, f, cur, min(nxt, b), step))
                cur = nxt
            for task in ans:
                acc, start_time, end_time, left, right = task.result()
                res += acc
                logs.append((start_time, end_time, left, right))
    
    
    if log:
        fout = open("artifacts/log_integrate.txt", "w")
        for log in logs:
            print("Integrate from {} to {} for values from {} to {}\n".format(*log), file=fout)
        fout.close()
        
    return res

In [3]:
import math
import multiprocessing as mp
from time import time

cpu_num = mp.cpu_count()
fout = open('artifacts/medium.txt', 'w')
for jobs in range(1, 2 * cpu_num + 1):
    print(f'{jobs} threads:\n', file = fout)
    start = time()
    result = integrate_async(math.cos, 0, math.pi / 2, n_iter=1000000, n_jobs=jobs)
    print(f'Time: {time() - start} seconds', f'Result: {result}', file=fout, sep='\n', end='\n\n')

    print(f'{jobs} processes:\n', file = fout)
    start = time()
    result = integrate_async(math.cos, 0, math.pi / 2, n_iter=1000000, n_jobs=jobs, process=True)
    print(f'Time: {time() - start} seconds', f'Result: {result}', file=fout, sep='\n', end='\n\n')
    
fout.close()